Note: Run on the Picasso Kernel

In [ ]:
# Import dependencies.

import os as _os
import os.path as _ospath
import numpy as _np
import pandas as _pd
import yaml as _yaml
from picasso import io as _io
from picasso import postprocess as _postprocess
from scipy.spatial import cKDTree

In [ ]:
# Define folder and files for the pick files.

folder_picks = '/Users/abhinav/Library/CloudStorage/OneDrive-IndianInstituteofScience/Papers/zz_Msequences/Multiplexing/20250116/Yaml'  # Directory containing the pick positions.
pick_extn = '.yaml'                                                                                                                     # Pick information file extension.
pick_files = [f for f in _os.listdir(folder_picks) if f.endswith(pick_extn) and 'picks' in f] 
distance_threshold = 2
batch_size = 10000
max_iterations = 10000

In [ ]:
# Functions used in this script. (Adapted from picasso)

def read_pick_locations(pick_file):
    """Read pick coordinates, diameter, and shape from a Picasso .yaml pick file."""
    with open(pick_file, 'r') as f:
        regions = _yaml.safe_load(f)
    picks = _np.array(regions['Centers'])
    pick_size = regions['Diameter']
    pick_shape = regions.get('Shape', 'Circle')
    return picks, pick_size, pick_shape

def is_far_enough(point, existing_points, threshold):
    """Check if a point is at least threshold distance away from existing points."""
    if len(existing_points) == 0:
        return True
    dists = _np.linalg.norm(existing_points - point, axis=1)
    return _np.all(dists >= threshold)


def generate_random_picks(bounds, existing_picks, n_new, threshold, max_tries=100000):
    """Generate n_new random picks within bounds, far from existing picks and each other."""
    new_picks = []
    xmin, xmax, ymin, ymax = bounds
    attempts = 0

    while len(new_picks) < n_new and attempts < max_tries:
        attempts += 1
        # Random candidate
        candidate = _np.array([
            _np.random.uniform(xmin, xmax),
            _np.random.uniform(ymin, ymax)
        ])
        # Check distance from both existing and already accepted picks
        if is_far_enough(candidate, existing_picks, threshold) and \
           is_far_enough(candidate, _np.array(new_picks), threshold):
            new_picks.append(candidate)

    if len(new_picks) < n_new:
        print(f"⚠️ Only generated {len(new_picks)} out of {n_new} desired picks after {attempts} attempts.")
    else:
        print(f"✅ Generated {len(new_picks)} picks successfully.")

    return _np.array(new_picks)

In [ ]:
# Collect all existing picks
pick_files = sorted([f for f in _os.listdir(folder_picks) if f.endswith(pick_extn) and 'picks' in f])
pick_paths = [_ospath.join(folder_picks, f) for f in pick_files]

all_picks = []
total_picks = 0
for pf in pick_paths:
    picks, pick_size, pick_shape = read_pick_locations(pf)
    all_picks.append(picks)
    total_picks += len(picks)

# Compute average number of picks
n_files = len(all_picks)
avg_picks = int(round(total_picks / n_files))
existing_picks = _np.vstack(all_picks)
print(f"Average number of picks per file: {avg_picks}")

# Build KDTree for fast distance queries
tree_existing = cKDTree(existing_picks)

# Determine bounding box
x_min, y_min = existing_picks.min(axis=0)
x_max, y_max = existing_picks.max(axis=0)
bounds = (x_min, x_max, y_min, y_max)
print(f"Bounding box: x=[{x_min:.1f}, {x_max:.1f}], y=[{y_min:.1f}, {y_max:.1f}]")

In [ ]:
# Initialize new picks list and KDTree for them too
new_picks = []
tree_new = None

iteration = 0
while len(new_picks) < avg_picks and iteration < max_iterations:
    iteration += 1

    # Random batch of candidate points
    candidates = _np.column_stack((
        _np.random.uniform(bounds[0], bounds[1], batch_size),
        _np.random.uniform(bounds[2], bounds[3], batch_size)
    ))

    # Check distance to existing picks
    d_exist, _ = tree_existing.query(candidates, distance_upper_bound=distance_threshold)
    mask_far_from_existing = d_exist > distance_threshold

    if tree_new is not None and len(new_picks) > 0:
        # Also check distance to already accepted new picks
        d_new, _ = tree_new.query(candidates, distance_upper_bound=distance_threshold)
        mask_far_from_new = d_new > distance_threshold
        mask_valid = mask_far_from_existing & mask_far_from_new
    else:
        mask_valid = mask_far_from_existing

    valid_points = candidates[mask_valid]

    # Add to list
    if len(valid_points) > 0:
        new_picks.append(valid_points)
        combined = _np.vstack(new_picks)
        tree_new = cKDTree(combined)

    print(f"Iteration {iteration}: {len(valid_points)} valid picks found, total {len(new_picks) and len(combined)}")

    # Stop if we reached target count
    if len(combined) >= avg_picks:
        break

# Finalize
new_picks = combined[:avg_picks]
print(f"✅ Generated {len(new_picks)} random picks at least {distance_threshold} nm from any existing pick.")

In [ ]:
# --- Save to YAML ---
output_yaml = {
    'Centers': new_picks.tolist(),
    'Diameter': pick_size,
    'Shape': pick_shape,
}

out_file = _ospath.join(folder_picks, 'random_average_picks.yaml')
with open(out_file, 'w') as f:
    _yaml.dump(output_yaml, f)

print(f"💾 Saved to {out_file}")